![(book cover)](https://covers.oreillystatic.com/images/0636920167433/cat.gif "(book cover)")
# "Programming Quantum Computers" by O'Reilly Media -  [book info](http://shop.oreilly.com/product/0636920167433.do)  - [all code samples](https://oreilly-qc.github.io)

## Code samples for Chapter 7
These code samples were written by Mariia Mykhailova.

### Example 7-1: Using QFT to distinguish between three states

In [1]:
// Example 7-1: Using QFT to distinguish between three states

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Diagnostics;

operation PrepareState (register : Qubit[], state : String) : Unit {
    ApplyToEach(H, register);
    if (state == "A") {
        Z(register[0]);
    } elif (state == "B") {
        S(register[0]);
        Z(register[1]);
    } else {
        T(register[0]);
        S(register[1]);
        Z(register[2]);
    }
}

// An operation that is given a register in one of the states A, B or C
// and returns the letter of the state it recognized
operation ClassifyState (register : Qubit[]) : String {
    // Apply QFT
    // (Q# library implementation of QFT is inverse compared to the one used in the book,
    // so this code uses Adjoint QFTLE to match the desired effect).
    Adjoint QFTLE(LittleEndian(register));
    
    // Uncomment this line to see the amplitudes after QFT
    // DumpMachine();
    
    // Measure the register and interpret the results
    let frequency = MeasureInteger(LittleEndian(register));
    return frequency == 8 ? "A" | (frequency == 4 ? "B" | "C");
}

operation PrepareAndClassifyStates () : Unit {
    using (register = Qubit[4]) {
        for (inputState in ["A", "B", "C"]) {
            // Prepare the register in the given state
            PrepareState(register, inputState);
            
            // Uncomment this line to see the amplitudes of the prepared state
            // DumpMachine();
            
            // Classify the state
            let classifiedState = ClassifyState(register);
            Message($"State {inputState} recognized as {classifiedState}");
            
            // Make sure to reset qubits to the |0❭ state before reusing/releasing them
            ResetAll(register);
        }
    }
}

ClassifyState, PrepareAndClassifyStates, PrepareState

In [2]:
%simulate PrepareAndClassifyStates

State A recognized as A
State B recognized as B
State C recognized as C


()

### Example 7-2: QFT of simple QPU register signal

In [3]:
// Example 7-2: QFT of simple QPU register signal

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Diagnostics;

operation SimpleStateQFT () : Unit {
    using (register = Qubit[4]) {
        // use PrepareState operation from previous sample to prepare the state
        PrepareState(register, "C");
        
        // Observe the amplitudes of the prepared state
        DumpMachine();
        
        // Apply QFT
        Adjoint QFTLE(LittleEndian(register));
        
        // Observe the amplitudes of the state after QFT transformation
        DumpMachine();
        
        // Make sure to reset qubits to the |0❭ state before releasing them
        ResetAll(register);
    }
}

SimpleStateQFT

In [4]:
%simulate SimpleStateQFT

|0⟩	0.25000000000000006 + 0𝑖
|1⟩	0.17677669529663692 + 0.17677669529663692𝑖
|2⟩	0 + 0.25000000000000006𝑖
|3⟩	-0.17677669529663692 + 0.17677669529663692𝑖
|4⟩	-0.25000000000000006 + 0𝑖
|5⟩	-0.17677669529663692 + -0.17677669529663692𝑖
|6⟩	0 + -0.25000000000000006𝑖
|7⟩	0.17677669529663692 + -0.17677669529663692𝑖
|8⟩	0.25000000000000006 + 0𝑖
|9⟩	0.17677669529663692 + 0.17677669529663692𝑖
|10⟩	0 + 0.25000000000000006𝑖
|11⟩	-0.17677669529663692 + 0.17677669529663692𝑖
|12⟩	-0.25000000000000006 + 0𝑖
|13⟩	-0.17677669529663692 + -0.17677669529663692𝑖
|14⟩	0 + -0.25000000000000006𝑖
|15⟩	0.17677669529663692 + -0.17677669529663692𝑖

|0⟩	-3.213662846257948E-33 + -5.7483679257327405E-18𝑖
|1⟩	8.865557782624204E-18 + 1.4149464739225804E-17𝑖
|2⟩	1.0000000000000007 + -5.4680469351266464E-17𝑖
|3⟩	-1.0525710568628527E-17 + 2.238296998364588E-17𝑖
|4⟩	-3.213662846257948E-33 + 5.7483679257327405E-18𝑖
|5⟩	1.699497766470593E-17 + 1.360611087640311E-17𝑖
|6⟩	2.0456837613538556E-17 + -7.298738002090358E-18𝑖
|7⟩	-7.2445927957825455E-19 + 8.493355598108097E-18𝑖
|8⟩	-2.9493129757812066E-33 + 5.748367925732734E-18𝑖
|9⟩	-1.6994977664705936E-17 + -1.9897832664958546E-17𝑖
|10⟩	8.243604496689538E-17 + 5.4680469351266464E-17𝑖
|11⟩	2.3962906865467994E-18 + -1.6634602057913134E-17𝑖
|12⟩	-2.9493129757812066E-33 + -5.748367925732734E-18𝑖
|13⟩	-8.865557782624212E-18 + -7.857742950670381E-18𝑖
|14⟩	-2.0456837613538556E-17 + 7.298738002090358E-18𝑖
|15⟩	8.853879161659971E-18 + -1.4241723523840828E-17𝑖

()

### Example 7-3: Square-wave QFT

In [5]:
// Example 7-3: Square-wave QFT

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Diagnostics;

operation SquareWaveQFT () : Unit {
    using (register = Qubit[4]) {
        let wavePeriod = 2;   // can range from 1 to 4
        // Prepare the state
        ApplyToEach(H, register);
        Z(register[wavePeriod - 1]);
        
        // Observe the amplitudes of the prepared state
        DumpMachine();
        
        // Apply QFT
        Adjoint QFTLE(LittleEndian(register));
        
        // Observe the amplitudes of the state after QFT transformation
        DumpMachine();
        
        // Make sure to reset qubits to the |0❭ state before releasing them
        ResetAll(register);
    }
}

SquareWaveQFT

In [6]:
%simulate SquareWaveQFT

|0⟩	0.25000000000000006 + 0𝑖
|1⟩	0.25000000000000006 + 0𝑖
|2⟩	-0.25000000000000006 + 0𝑖
|3⟩	-0.25000000000000006 + 0𝑖
|4⟩	0.25000000000000006 + 0𝑖
|5⟩	0.25000000000000006 + 0𝑖
|6⟩	-0.25000000000000006 + 0𝑖
|7⟩	-0.25000000000000006 + 0𝑖
|8⟩	0.25000000000000006 + 0𝑖
|9⟩	0.25000000000000006 + 0𝑖
|10⟩	-0.25000000000000006 + 0𝑖
|11⟩	-0.25000000000000006 + 0𝑖
|12⟩	0.25000000000000006 + 0𝑖
|13⟩	0.25000000000000006 + 0𝑖
|14⟩	-0.25000000000000006 + 0𝑖
|15⟩	-0.25000000000000006 + 0𝑖

|0⟩	-2.465190328815662E-32 + 0𝑖
|1⟩	3.4929992047024634E-18 + -2.3024732265279648E-18𝑖
|2⟩	-7.439369907072203E-33 + -3.0814879110195774E-33𝑖
|3⟩	-5.0240272582903655E-18 + 1.0730376532709257E-18𝑖
|4⟩	0.5000000000000003 + -0.5000000000000003𝑖
|5⟩	7.128599739155197E-18 + 5.558662090484767E-18𝑖
|6⟩	1.2763940850330464E-33 + 3.081487911019578E-33𝑖
|7⟩	9.122045114593084E-18 + 7.931519136418594E-18𝑖
|8⟩	2.465190328815662E-32 + 0𝑖
|9⟩	4.636420677379256E-18 + -3.445894699204763E-18𝑖
|10⟩	1.276394085033046E-33 + -3.081487911019579E-33𝑖
|11⟩	-3.1053926237913626E-18 + -6.82140557900367E-18𝑖
|12⟩	0.5000000000000003 + 0.5000000000000003𝑖
|13⟩	-1.5258019621236928E-17 + 1.8970583524797319E-19𝑖
|14⟩	-7.439369907072203E-33 + 3.081487911019578E-33𝑖
|15⟩	-9.926252325113694E-19 + -2.183151210685862E-18𝑖

()

### Example 7-4: Square-wave QFT on a qubyte

In [7]:
// Example 7-4: Square-wave QFT on a qubyte

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Diagnostics;

operation SquareWaveQFTQubyte () : Unit {
    using (register = Qubit[8]) {
        // Prepare the state
        ApplyToEach(H, register);
        Z(register[4]);
        
        // Observe the amplitudes of the prepared state
        DumpMachine();
        
        // Apply QFT
        Adjoint QFTLE(LittleEndian(register));
        
        // Observe the amplitudes of the state after QFT transformation
        DumpMachine();
        
        // Make sure to reset qubits to the |0❭ state before releasing them
        ResetAll(register);
    }
}

SquareWaveQFTQubyte

*(output of the next cell is not shown due to its size; run the cell to generate the output)*

In [ ]:
%simulate SquareWaveQFTQubyte

### Example 7-5: Converting frequency into state

In [9]:
// Example 7-5: Converting frequency into state

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Diagnostics;

operation FrequencyToState () : Unit {
    using (register = Qubit[4]) {
        // Write frequency to register
        X(register[0]);
        X(register[1]);
        
        // Observe the amplitudes of the prepared state
        DumpMachine();
        
        // Apply QFT
        QFTLE(LittleEndian(register));
        
        // Observe the amplitudes of the state after QFT transformation
        DumpMachine();
        
        // Make sure to reset qubits to the |0❭ state before releasing them
        ResetAll(register);
    }
}

FrequencyToState

In [10]:
%simulate FrequencyToState

|0⟩	0 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0 + 0𝑖
|3⟩	1 + 0𝑖
|4⟩	0 + 0𝑖
|5⟩	0 + 0𝑖
|6⟩	0 + 0𝑖
|7⟩	0 + 0𝑖
|8⟩	0 + 0𝑖
|9⟩	0 + 0𝑖
|10⟩	0 + 0𝑖
|11⟩	0 + 0𝑖
|12⟩	0 + 0𝑖
|13⟩	0 + 0𝑖
|14⟩	0 + 0𝑖
|15⟩	0 + 0𝑖

|0⟩	0.25000000000000006 + 0𝑖
|1⟩	0.09567085809127247 + 0.2309698831278217𝑖
|2⟩	-0.1767766952966369 + 0.17677669529663687𝑖
|3⟩	-0.2309698831278217 + -0.09567085809127243𝑖
|4⟩	0 + -0.25000000000000006𝑖
|5⟩	0.2309698831278217 + -0.09567085809127247𝑖
|6⟩	0.17677669529663687 + 0.1767766952966369𝑖
|7⟩	-0.09567085809127243 + 0.2309698831278217𝑖
|8⟩	-0.25000000000000006 + 0𝑖
|9⟩	-0.09567085809127247 + -0.2309698831278217𝑖
|10⟩	0.1767766952966369 + -0.17677669529663687𝑖
|11⟩	0.2309698831278217 + 0.09567085809127243𝑖
|12⟩	0 + 0.25000000000000006𝑖
|13⟩	-0.2309698831278217 + 0.09567085809127247𝑖
|14⟩	-0.17677669529663687 + -0.1767766952966369𝑖
|15⟩	0.09567085809127243 + -0.2309698831278217𝑖

()

### Example 7-6: Prepare a state with varying ampitudes using QFT

In [11]:
// Example 7-6: Prepare a state with varying ampitudes using QFT

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Diagnostics;

operation PrepareStateWithQFT () : Unit {
    using (register = Qubit[4]) {
        // Write the frequencies we want to register (a superposition of |1❭ and |15❭)
        H(register[0]);
        ApplyToEach(CNOT(Head(register), _), Rest(register));
        X(register[1]);
        CNOT(register[1], register[0]);
        X(register[1]);
        
        // Observe the amplitudes of the prepared state
        DumpMachine();
        
        // Apply QFT
        QFTLE(LittleEndian(register));
        
        // Observe the amplitudes of the state after QFT transformation
        DumpMachine();
        
        // Make sure to reset qubits to the |0❭ state before releasing them
        ResetAll(register);
    }
}

PrepareStateWithQFT

In [12]:
%simulate PrepareStateWithQFT

|0⟩	0 + 0𝑖
|1⟩	0.7071067811865476 + 0𝑖
|2⟩	0 + 0𝑖
|3⟩	0 + 0𝑖
|4⟩	0 + 0𝑖
|5⟩	0 + 0𝑖
|6⟩	0 + 0𝑖
|7⟩	0 + 0𝑖
|8⟩	0 + 0𝑖
|9⟩	0 + 0𝑖
|10⟩	0 + 0𝑖
|11⟩	0 + 0𝑖
|12⟩	0 + 0𝑖
|13⟩	0 + 0𝑖
|14⟩	0 + 0𝑖
|15⟩	0.7071067811865476 + 0𝑖

|0⟩	0.35355339059327384 + 0𝑖
|1⟩	0.3266407412190942 + -2.2550372918288193E-17𝑖
|2⟩	0.25 + 3.0292209705021295E-17𝑖
|3⟩	0.13529902503654928 + 6.073994565574205E-18𝑖
|4⟩	0 + -1.1496735851465466E-17𝑖
|5⟩	-0.13529902503654928 + 6.073994565574205E-18𝑖
|6⟩	-0.25 + -1.7089521644154808E-17𝑖
|7⟩	-0.3266407412190942 + 3.4644436297661506E-17𝑖
|8⟩	-0.35355339059327384 + 0𝑖
|9⟩	-0.3266407412190942 + 2.2550372918288193E-17𝑖
|10⟩	-0.25 + -3.0292209705021295E-17𝑖
|11⟩	-0.13529902503654928 + -6.073994565574205E-18𝑖
|12⟩	0 + 1.1496735851465466E-17𝑖
|13⟩	0.13529902503654928 + -6.073994565574205E-18𝑖
|14⟩	0.25 + 1.7089521644154808E-17𝑖
|15⟩	0.3266407412190942 + -3.4644436297661506E-17𝑖

()

### Example 7-7: QFT frequency manipulation

In [13]:
// Example 7-7: QFT frequency manipulation

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Diagnostics;

operation QFTFrequencyManipulation () : Unit {
    using (register = Qubit[4]) {
        // Write frequency to register
        X(register[1]);
        // Uncomment the line below to observe the amplitudes of the prepared state
        // DumpMachine();
        
        // Apply QFT to prepare our initial state
        let registerLE = LittleEndian(register);
        QFTLE(registerLE);
        // Observe the amplitudes of the initial state
        DumpMachine();
        
        // Move back to frequency space
        Adjoint QFTLE(registerLE);
        
        // Increase the frequency of the signal
        IncrementByInteger(1, registerLE);
        
        // Move back to signal space
        QFTLE(registerLE);
        // Observe the amplitudes of the resulting state
        DumpMachine();
        
        // Make sure to reset qubits to the |0❭ state before releasing them
        ResetAll(register);
    }
}

QFTFrequencyManipulation

In [14]:
%simulate QFTFrequencyManipulation

|0⟩	0.25000000000000006 + 0𝑖
|1⟩	0.17677669529663692 + 0.17677669529663692𝑖
|2⟩	0 + 0.25000000000000006𝑖
|3⟩	-0.17677669529663692 + 0.17677669529663692𝑖
|4⟩	-0.25000000000000006 + 0𝑖
|5⟩	-0.17677669529663692 + -0.17677669529663692𝑖
|6⟩	0 + -0.25000000000000006𝑖
|7⟩	0.17677669529663692 + -0.17677669529663692𝑖
|8⟩	0.25000000000000006 + 0𝑖
|9⟩	0.17677669529663692 + 0.17677669529663692𝑖
|10⟩	0 + 0.25000000000000006𝑖
|11⟩	-0.17677669529663692 + 0.17677669529663692𝑖
|12⟩	-0.25000000000000006 + 0𝑖
|13⟩	-0.17677669529663692 + -0.17677669529663692𝑖
|14⟩	0 + -0.25000000000000006𝑖
|15⟩	0.17677669529663692 + -0.17677669529663692𝑖

|0⟩	0.2500000000000003 + -1.2871883064922752E-17𝑖
|1⟩	0.09567085809127256 + 0.23096988312782196𝑖
|2⟩	-0.17677669529663703 + 0.17677669529663703𝑖
|3⟩	-0.23096988312782196 + -0.09567085809127256𝑖
|4⟩	-8.803152104593907E-18 + -0.2500000000000003𝑖
|5⟩	0.23096988312782196 + -0.09567085809127258𝑖
|6⟩	0.17677669529663703 + 0.17677669529663703𝑖
|7⟩	-0.09567085809127253 + 0.2309698831278219𝑖
|8⟩	-0.2500000000000003 + 4.3488053043606624E-17𝑖
|9⟩	-0.0956708580912726 + -0.23096988312782196𝑖
|10⟩	0.176776695296637 + -0.1767766952966371𝑖
|11⟩	0.23096988312782193 + 0.09567085809127254𝑖
|12⟩	3.941932208327777E-17 + 0.2500000000000003𝑖
|13⟩	-0.23096988312782196 + 0.09567085809127258𝑖
|14⟩	-0.17677669529663706 + -0.176776695296637𝑖
|15⟩	0.09567085809127256 + -0.23096988312782196𝑖

()

### Example 7-8: Rotating phases by different angles

In [15]:
// Example 7-8: Rotating phases by different angles

open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;

operation RotatingPhases () : Unit {
    // Rotate k-th basis state by k*20 degrees
    let phi = 20.0 / 180.0 * PI();  // the rotation angle in radians
    
    using (register = Qubit[4]) {
        // Put the register in superposition of all basis states to see all results at once
        ApplyToEach(H, register);
        // DumpMachine();
        
        for (i in 0 .. 3) {
            R1(phi * IntAsDouble(1 <<< i), register[i]);
        }
        
        // Observe the amplitudes of the resulting state
        DumpMachine();
        
        // Make sure to reset qubits to the |0❭ state before releasing them
        ResetAll(register);
    }
}

RotatingPhases

In [16]:
%simulate RotatingPhases

|0⟩	0.25 + 1.3877787807814457E-17𝑖
|1⟩	0.2349231551964771 + 0.08550503583141722𝑖
|2⟩	0.19151111077974456 + 0.1606969024216348𝑖
|3⟩	0.125 + 0.21650635094610965𝑖
|4⟩	0.04341204441673261 + 0.246201938253052𝑖
|5⟩	-0.04341204441673256 + 0.24620193825305203𝑖
|6⟩	-0.125 + 0.21650635094610976𝑖
|7⟩	-0.19151111077974448 + 0.16069690242163487𝑖
|8⟩	-0.23492315519647708 + 0.08550503583141719𝑖
|9⟩	-0.25 + 2.0816681711721685E-17𝑖
|10⟩	-0.23492315519647716 + -0.08550503583141716𝑖
|11⟩	-0.19151111077974453 + -0.16069690242163479𝑖
|12⟩	-0.12500000000000006 + -0.21650635094610965𝑖
|13⟩	-0.043412044416732645 + -0.246201938253052𝑖
|14⟩	0.04341204441673253 + -0.24620193825305206𝑖
|15⟩	0.12499999999999992 + -0.2165063509461097𝑖

()